<a href="https://colab.research.google.com/github/MarylouBer/MLD_Data_Engineering/blob/main/comparenormalandresendqueues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import boto3
import botocore
from tabulate import tabulate

# Use the "saml" profile
session = boto3.Session(profile_name="saml", region_name="eu-central-1")
sqs = session.client("sqs")

country_codes = ["MY"]  # Replace with actual country codes

# List of attributes to exclude
exclude_attributes = [
    "ApproximateNumberOfMessagesNotVisible",
    "ApproximateNumberOfMessagesDelayed",
    "ApproximateNumberOfMessages",
    "LastModifiedTimestamp",
    "CreatedTimestamp"
]

def get_queue_details(queue_name):
    try:
        response = sqs.get_queue_url(QueueName=queue_name)
        queue_url = response["QueueUrl"]

        # Get all attributes and filter out the excluded ones
        all_attributes = sqs.get_queue_attributes(QueueUrl=queue_url, AttributeNames=["All"])["Attributes"]
        filtered_attributes = {key: value for key, value in all_attributes.items() if key not in exclude_attributes}

        # Extract tags
        tags = sqs.list_queue_tags(QueueUrl=queue_url).get("Tags", {})

        # Extract specific details
        details = {
            "Queue Type": "FIFO" if all_attributes.get("FifoQueue", "false") == "true" else "Standard",
            "Encryption Key": all_attributes.get("KmsMasterKeyId", "Not Configured"),
            "Encryption Reuse Period (s)": all_attributes.get("KmsDataKeyReusePeriodSeconds", "Not Set"),
            "Redrive Allow Policy": all_attributes.get("RedriveAllowPolicy", "Not Configured")
        }

        return filtered_attributes, tags, details, queue_url
    except botocore.exceptions.NoCredentialsError:
        print(f"Error: AWS credentials not found. Try running `aws configure` or logging in via SSO.")
        return None, None, None, None
    except Exception as e:
        print(f"Error retrieving queue {queue_name}: {str(e)}")
        return None, None, None, None

for country in country_codes:
    normal_queue_name = f"XX_{country}_PRD"
    resend_queue_name = f"XX_{country}_RESEND_PRD"

    normal_attributes, normal_tags, normal_details, normal_queue_url = get_queue_details(normal_queue_name)
    resend_attributes, resend_tags, resend_details, resend_queue_url = get_queue_details(resend_queue_name)

    if not normal_attributes or not resend_attributes:
        continue

    # Compare attributes
    all_keys = set(normal_attributes.keys()).union(set(resend_attributes.keys()))
    differences = []

    for key in all_keys:
        normal_value = normal_attributes.get(key, "Not Found")
        resend_value = resend_attributes.get(key, "Not Found")
        status = "Same" if normal_value == resend_value else "Different"
        differences.append([key, status, normal_value, resend_value])  # Comparison column after Attribute

    # Compare specific details
    for key in normal_details.keys():
        normal_value = normal_details[key]
        resend_value = resend_details[key]
        status = "Same" if normal_value == resend_value else "Different"
        differences.append([key, status, normal_value, resend_value])  # Comparison column after Attribute

    # Compare tags
    if normal_tags or resend_tags:  # There are tags in at least one queue
        all_tag_keys = set(normal_tags.keys()).union(set(resend_tags.keys()))
        for key in all_tag_keys:
            normal_value = normal_tags.get(key, "Not Found")
            resend_value = resend_tags.get(key, "Not Found")
            status = "Same" if normal_value == resend_value else "Different"
            differences.append([f"Tag: {key}", status, normal_value, resend_value])
    else:
        # If no tags exist in either queue, add a placeholder row and mark them as "Same"
        differences.append(["Tag", "Same", "No Tags Present", "No Tags Present"])


    # Write results to a Markdown file
    output_file = f"comparison_{country}_PRD.md"
    with open(output_file, "w") as file:
        file.write(f"\n **Comparison for {country} ({normal_queue_name} vs {resend_queue_name})**\n")
        file.write("```markdown\n")
        file.write(tabulate(differences, headers=["Attribute", "Comparison", "Normal Queue", "Resend Queue"], tablefmt="pipe"))
        file.write("\n```\n")

    print(f"Results for {country} have been written to {output_file}.")
